In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import cv2
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [2]:
from PIL import Image  # Import PIL

class ChestXRayDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = ['covid', 'pneumonia', 'normal']
        self.data = []

        for label, category in enumerate(self.classes):
            category_path = os.path.join(root_dir, category)
            for img_name in os.listdir(category_path):
                img_path = os.path.join(category_path, img_name)
                self.data.append((img_path, label))
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        img_path, label = self.data[index]
        
        # Load image using PIL instead of OpenCV
        image = Image.open(img_path).convert("RGB") 
        
        if self.transform:
            image = self.transform(image)  # Apply transformations
        
        return image, label


In [3]:

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load datasets
train_dataset = ChestXRayDataset("/kaggle/input/dataset/dataset/train", transform=transform)
test_dataset = ChestXRayDataset("/kaggle/input/dataset/dataset/test", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# VGG-16 Model
vgg16 = models.vgg16(pretrained=True)
vgg16.classifier[6] = nn.Linear(4096, 3)
vgg16 = vgg16.to(device)

# DenseNet-201 Model
densenet201 = models.densenet201(pretrained=True)
densenet201.classifier = nn.Linear(1920, 3)
densenet201 = densenet201.to(device)

# EfficientNet-B0 Model
efficientnet_b0 = models.efficientnet_b0(pretrained=True)
efficientnet_b0.classifier[1] = nn.Linear(1280, 3)
efficientnet_b0 = efficientnet_b0.to(device)

# Loss and Optimizers
criterion = nn.CrossEntropyLoss()
optimizer_vgg16 = optim.SGD(vgg16.parameters(), lr=0.0001, momentum=0.9)
optimizer_densenet = optim.SGD(densenet201.parameters(), lr=0.0001, momentum=0.9)
optimizer_efficientnet = optim.SGD(efficientnet_b0.parameters(), lr=0.0001, momentum=0.9)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:02<00:00, 210MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. 

In [5]:
def train_model(model, train_loader, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in tqdm(train_loader):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}, Accuracy: {100 * correct/total:.2f}%")

In [6]:
# Train the models
train_model(vgg16, train_loader, optimizer_vgg16, num_epochs=10)

100%|██████████| 167/167 [01:22<00:00,  2.03it/s]


Epoch [1/10], Loss: 0.43186915514176477, Accuracy: 83.33%


100%|██████████| 167/167 [01:20<00:00,  2.08it/s]


Epoch [2/10], Loss: 0.20751014197925607, Accuracy: 92.38%


100%|██████████| 167/167 [01:20<00:00,  2.08it/s]


...

Epoch [8/10], Loss: 0.08860117198709777, Accuracy: 96.70%


100%|██████████| 167/167 [01:20<00:00,  2.07it/s]


Epoch [9/10], Loss: 0.07653075311065255, Accuracy: 97.37%


100%|██████████| 167/167 [01:20<00:00,  2.07it/s]

Epoch [10/10], Loss: 0.07092269771721237, Accuracy: 97.41%


In [7]:
train_model(densenet201, train_loader, optimizer_densenet, num_epochs=10)

100%|██████████| 167/167 [01:27<00:00,  1.92it/s]


Epoch [1/10], Loss: 0.5284478572492828, Accuracy: 77.85%


100%|██████████| 167/167 [01:26<00:00,  1.93it/s]


Epoch [2/10], Loss: 0.2815259611178301, Accuracy: 92.02%


100%|██████████| 167/167 [01:26<00:00,  1.92it/s]


...

Epoch [8/10], Loss: 0.09381818237664279, Accuracy: 97.09%


100%|██████████| 167/167 [01:26<00:00,  1.92it/s]


Epoch [9/10], Loss: 0.09145328775048256, Accuracy: 97.18%


100%|██████████| 167/167 [01:26<00:00,  1.92it/s]

Epoch [10/10], Loss: 0.08904129633133462, Accuracy: 97.20%


In [8]:
train_model(efficientnet_b0, train_loader, optimizer_efficientnet, num_epochs=10)

100%|██████████| 167/167 [00:53<00:00,  3.15it/s]


Epoch [1/10], Loss: 0.8103018110383771, Accuracy: 66.51%


100%|██████████| 167/167 [00:53<00:00,  3.10it/s]


Epoch [2/10], Loss: 0.5389805750218694, Accuracy: 79.11%


100%|██████████| 167/167 [00:51<00:00,  3.22it/s]


...

Epoch [8/10], Loss: 0.23963061853975592, Accuracy: 91.98%


100%|██████████| 167/167 [00:50<00:00,  3.28it/s]


Epoch [9/10], Loss: 0.2229886684499815, Accuracy: 92.96%


100%|██████████| 167/167 [00:51<00:00,  3.24it/s]

Epoch [10/10], Loss: 0.210157166028808, Accuracy: 93.04%


In [9]:
# Save the models
torch.save(vgg16.state_dict(), "vgg16_model.pth")
torch.save(densenet201.state_dict(), "densenet201_model.pth")
torch.save(efficientnet_b0.state_dict(), "efficientnet_b0_model.pth")
print("Models saved successfully!")

Models saved successfully!


In [10]:
# Load models
vgg16.load_state_dict(torch.load("vgg16_model.pth"))
vgg16.to(device).eval()

densenet201.load_state_dict(torch.load("densenet201_model.pth"))
densenet201.to(device).eval()

efficientnet_b0.load_state_dict(torch.load("efficientnet_b0_model.pth"))
efficientnet_b0.to(device).eval()

ensemble_models = [vgg16, densenet201, efficientnet_b0]
print("Models loaded successfully!")


<ipython-input-10-35650c412449>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vgg16.load_state_dict(torch.load("vgg16_model.pth"))
<ipython-input-10-35650c412449>:5: Futu

Models loaded successfully!


<ipython-input-10-35650c412449>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  efficientnet_b0.load_state_dict(torch.load("efficientnet_b0_model.pth"))


In [11]:
class_labels = ["Covid", "Pneumonia", "Normal"]

def predict_image(image_path, models):
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)  # Add batch dimension
    image = image.to(device)

    models = [model.eval() for model in models]

    with torch.no_grad():
        avg_output = sum(model(image) for model in models) / len(models)
        _, predicted_class = avg_output.max(1)

    return class_labels[predicted_class.item()]

# Test with a single image
image_path = "/kaggle/input/dataset/dataset/test/pneumonia/person27_bacteria_137.jpeg"  # Replace with your X-ray image path
prediction = predict_image(image_path, ensemble_models)
print(f"Predicted Disease: {prediction}")


Predicted Disease: Pneumonia


In [12]:
def evaluate_ensemble(models, test_loader):
    models = [model.eval() for model in models]
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(test_loader):
            images, labels = images.to(device), labels.to(device)

            avg_output = sum(model(images) for model in models) / len(models)
            _, predicted = avg_output.max(1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    acc = 100 * correct / total
    print(f"Ensemble Model Accuracy: {acc:.2f}%")
    print(classification_report(all_labels, all_preds, target_names=class_labels))

# Run evaluation
evaluate_ensemble(ensemble_models, test_loader)


100%|██████████| 4/4 [00:02<00:00,  1.39it/s]

Ensemble Model Accuracy: 97.50%
              precision    recall  f1-score   support

       Covid       1.00      0.92      0.96        25
   Pneumonia       0.94      1.00      0.97        30
      Normal       0.98      0.98      0.98        65

    accuracy                           0.97       120
   macro avg       0.97      0.97      0.97       120
weighted avg       0.98      0.97      0.97       120

